In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# plyaer data load 및 정리(프로필)
player_df = pd.read_csv('../data/new/players_all.csv')
player_df = player_df.rename(columns={'Unnamed: 0':'Id'}).drop(['No.', 'Id'], axis=1)

In [3]:
# 1xbet data load 및 정리(스텟)
xbet_df = pd.read_csv('../data/new/1xbet_all.csv')
xbet_df = xbet_df.drop(['Team','Age','Position'], axis=1).rename(columns={'Drb_x':'Drb_Off', 'Drb_y':'Drb_Def'})

In [4]:
# understat data load 및 정리(스텟)
understat_df = pd.read_csv('../data/new/understat_all.csv')
understat_df = understat_df.drop(['No','Team'], axis=1)

In [5]:
# capology load(연봉)
capology_df = pd.read_csv('../data/new/capology_all.csv')

In [6]:
# 프로필 + 연봉
eda_df = player_df.merge(capology_df[['Weekly Salary','Base Salary','ADJ Salary','Name', 'year']], how='left', left_on=['Name', 'year'], right_on=['Name', 'year'])
eda_df = eda_df[~eda_df.duplicated(['year', 'Name'], keep='first')] # 중복 제거

In [7]:
# 프로필 + 연봉 + 스텟(understat)
eda_df = eda_df.merge(understat_df, how='left', left_on=['Name', 'year'], right_on=['Name', 'year'])

In [8]:
# 프로필 + 연봉 + 스텟(understat) + 스텟(1xbet)
eda_df = eda_df.merge(
    xbet_df[[
        'Name', 'year', 'SpG', 'KeyP', 'Drb_Off', 'Fouled', 'Off',
        'Disp', 'UnsTch', 'Rating', 'Tackles', 'Inter','Fouls',
        'Offsides', 'Clear', 'Drb_Def', 'Blocks', 'AvgP', 'PS%']],
    how='left',
    left_on=['Name', 'year'],
    right_on=['Name', 'year']
)

In [9]:
eda_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4756 entries, 0 to 4755
Data columns (total 42 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           4756 non-null   int64  
 1   Name           4756 non-null   object 
 2   Age            4756 non-null   int64  
 3   Team           4756 non-null   object 
 4   Position       4756 non-null   object 
 5   Weekly Salary  4598 non-null   float64
 6   Base Salary    4598 non-null   float64
 7   ADJ Salary     4598 non-null   float64
 8   Apps           4744 non-null   float64
 9   Min            4744 non-null   float64
 10  G              4744 non-null   float64
 11  NPG            4744 non-null   float64
 12  A              4744 non-null   float64
 13  xG             4744 non-null   float64
 14  NPxG           4744 non-null   float64
 15  xA             4744 non-null   float64
 16  xGChain        4744 non-null   float64
 17  xGBuildup      4744 non-null   float64
 18  xG90    

In [10]:
eda_df = eda_df.dropna()

In [12]:
round(eda_df.describe(), 2)

,year,Age,Weekly Salary,Base Salary,ADJ Salary,Apps,Min,G,NPG,A,...,Rating,Tackles,Inter,Fouls,Offsides,Clear,Drb_Def,Blocks,AvgP,PS%
count,4588.00,4588.00,4588.00,4588.00,4588.00,4588.00,4588.00,4588.00,4588.00,4588.00,...,4588.00,4588.00,4588.00,4588.00,4588.00,4588.00,4588.00,4588.00,4588.00,4588.00
mean,2017.93,25.91,53921.12,2803898.52,3076834.48,20.58,1466.58,1.96,1.81,1.39,...,6.65,1.12,0.78,0.70,0.13,1.53,0.61,0.22,29.00,76.91
std,2.60,4.21,52782.05,2744666.44,2942732.53,11.49,1032.21,3.52,3.18,2.19,...,0.36,0.84,0.70,0.46,0.23,1.75,0.48,0.29,16.44,12.42
min,2014.00,15.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,...,4.95,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2016.00,23.00,20000.00,1040000.00,1252784.00,11.00,532.50,0.00,0.00,0.00,...,6.40,0.50,0.18,0.38,0.00,0.33,0.25,0.00,17.00,72.73
50%,2018.00,26.00,40000.00,2080000.00,2301310.00,22.00,1390.00,1.00,1.00,1.00,...,6.65,1.00,0.65,0.68,0.00,0.86,0.53,0.11,27.29,79.32
75%,2020.00,29.00,70000.00,3640000.00,4071549.00,31.00,2326.25,2.00,2.00,2.00,...,6.88,1.63,1.20,1.00,0.15,2.09,0.88,0.33,38.44,84.32
max,2022.00,41.00,600000.00,31200000.00,33671679.00,38.00,3420.00,36.00,31.00,20.00,...,8.34,7.00,4.22,4.00,2.00,11.50,4.00,2.05,90.41,100.00


In [16]:
eda_df['Age Lev'] = eda_df['Age'].apply(lambda age: '<25' if age < 25 else '<30' if age <30 else '<35' if age<35 else '>35')

In [20]:
eda_df.tail()

,year,Name,Age,Team,Position,Weekly Salary,Base Salary,ADJ Salary,Apps,Min,...,Tackles,Inter,Fouls,Offsides,Clear,Drb_Def,Blocks,AvgP,PS%,Age Lev
4745,2022,Alex Mighten,20,Nottingham Forest,Forward,3462.0,180000.0,180000.0,1.0,5.0,...,0.00,0.0,0.0,0.0,0.0,1.0,0.0,1.00,100.000000,<25
4746,2022,Jamal Lewis,24,Newcastle,Defender,38000.0,1976000.0,1976000.0,2.0,4.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,7.00,85.714286,<25
4747,2022,Maximo Perrone,19,Man City,Midfielder,30000.0,1560000.0,1560000.0,1.0,17.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,20.00,90.000000,<25
4750,2022,Nathaniel Chalobah,27,Fulham,Midfielder,35000.0,1820000.0,1820000.0,4.0,10.0,...,0.25,0.0,0.5,0.0,0.5,0.0,0.0,1.25,80.000000,<30
4754,2022,Shane Duffy,30,Fulham,Defender,40000.0,2080000.0,2080000.0,5.0,16.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,4.00,80.000000,<35


In [39]:
static_df = eda_df[eda_df['Min']>1000].groupby(['Position', 'Age Lev']).mean(numeric_only=True)
static_df

year        Age  Weekly Salary   Base Salary  \
Position   Age Lev                                                        
Defender   <25      2018.342020  22.416938   46630.543974  2.424788e+06   
           <30      2017.963783  26.885312   56289.541247  2.927056e+06   
           <35      2017.865741  31.333333   59561.879630  3.097218e+06   
           >35      2017.625000  35.750000   47937.500000  2.492750e+06   
Forward    <25      2018.206897  22.133005   71215.290640  3.703195e+06   
           <30      2017.790441  26.819853   81859.584559  4.256699e+06   
           <35      2018.687500  31.125000  100999.525000  5.251975e+06   
           >35      2019.000000  35.500000  178750.000000  9.295000e+06   
Goalkeeper <25      2018.437500  22.781250   58671.875000  3.050938e+06   
           <30      2017.806452  27.365591   67001.440860  3.484075e+06   
           <35      2018.044118  31.838235   70160.926471  3.648368e+06   
           >35      2018.315789  35.736842   53421.052632  2.777895e+06   
Midfielder <25      2018.222561  22.338415   54845.692073  2.851976e+06   
           <30      2017.807229  26.845382   69868.475904  3.633161e+06   
           <35      2017.883562  31.157534   83065.465753  4.319404e+06   
           >35      2018.200000  35.200000   90000.000000  4.680000e+06   

                      ADJ Salary       Apps          Min         G       NPG  \
Position   Age Lev                                                             
Defender   <25      2.636454e+06  25.762215  2104.214984  0.866450  0.866450   
           <30      3.214299e+06  26.722334  2228.742455  1.042254  1.028169   
           <35      3.420586e+06  26.458333  2208.657407  0.990741  0.967593   
           >35      2.695122e+06  24.625000  1946.000000  0.875000  0.875000   
Forward    <25      4.047246e+06  29.955665  2005.438424  7.177340  6.743842   
           <30      4.711195e+06  29.875000  2078.444853  8.408088  7.566176   
           <35      5.692078e+06  29.050000  1925.825000  8.137500  7.062500   
           >35      9.411844e+06  31.250000  1787.750000  8.250000  7.500000   
Goalkeeper <25      3.421782e+06  30.031250  2692.656250  0.000000  0.000000   
           <30      3.842430e+06  30.419355  2725.086022  0.010753  0.010753   
           <35      3.967441e+06  28.647059  2565.779412  0.000000  0.000000   
           >35      2.999916e+06  29.421053  2628.736842  0.000000  0.000000   
Midfielder <25      3.120253e+06  28.756098  2076.576220  3.164634  2.987805   
           <30      4.014345e+06  28.971888  2109.969880  2.759036  2.473896   
           <35      4.725229e+06  27.869863  2074.513699  1.938356  1.664384   
           >35      5.126010e+06  26.600000  1719.800000  0.600000  0.600000   

                           A  ...    Rating   Tackles     Inter     Fouls  \
Position   Age Lev            ...                                           
Defender   <25      1.319218  ...  6.823730  1.727525  1.385161  0.782325   
           <30      1.241449  ...  6.841142  1.621968  1.435161  0.798713   
           <35      0.995370  ...  6.804812  1.480141  1.481161  0.741381   
           >35      0.562500  ...  6.773137  1.302689  1.334605  0.682168   
Forward    <25      3.290640  ...  6.843510  0.808887  0.369946  0.856924   
           <30      3.371324  ...  6.861586  0.767153  0.350592  0.950121   
           <35      2.937500  ...  6.758889  0.631384  0.235737  0.861372   
           >35      2.750000  ...  6.806698  0.187400  0.066022  0.838547   
Goalkeeper <25      0.031250  ...  6.619492  0.023665  0.022129  0.018734   
           <30      0.139785  ...  6.685359  0.021120  0.029652  0.017939   
           <35      0.044118  ...  6.658262  0.023629  0.022669  0.013478   
           >35      0.052632  ...  6.617908  0.018727  0.037163  0.027904   
Midfielder <25      2.920732  ...  6.840155  1.636343  0.948094  0.959290   
           <30      2.672691  ...  6.797229  1.645536  1.012699  0.963996   
   

In [35]:
cond1 = eda_df['Name']=='Wayne Rooney'
cond2 = eda_df['year']==2017
rooney2017 = eda_df[ cond1 & cond2]

In [37]:
rooney2017['Position']

1890    Forward
Name: Position, dtype: object

In [36]:
rooney2017['Age Lev']

1890    <35
Name: Age Lev, dtype: object

In [70]:
(rooney2017['Position'].values[0],rooney2017['Age Lev'].values[0])

('Forward', '<35')

In [79]:
static_df.loc[(rooney2017['Position'].values[0],rooney2017['Age Lev'].values[0])]

year             2.018688e+03
Age              3.112500e+01
Weekly Salary    1.009995e+05
Base Salary      5.251975e+06
ADJ Salary       5.692078e+06
Apps             2.905000e+01
Min              1.925825e+03
G                8.137500e+00
NPG              7.062500e+00
A                2.937500e+00
xG               8.277375e+00
NPxG             7.268875e+00
xA               2.897375e+00
xGChain          1.104587e+01
xGBuildup        3.030250e+00
xG90             3.830000e-01
NPxG90           3.435000e-01
xA90             1.335000e-01
xG90+xA90        5.168750e-01
NPxG90+xA90      4.766250e-01
xGChain90        5.155000e-01
xGBuildup90      1.420000e-01
SpG              1.784074e+00
KeyP             8.569847e-01
Drb_Off          6.201139e-01
Fouled           8.695750e-01
Off              5.175349e-01
Disp             1.065322e+00
UnsTch           1.699707e+00
Rating           6.758889e+00
Tackles          6.313844e-01
Inter            2.357368e-01
Fouls            8.613720e-01
Offsides  

In [82]:
r = pd.DataFrame(data=static_df.loc[(rooney2017['Position'].values[0],rooney2017['Age Lev'].values[0])]).T
r

,,year,Age,Weekly Salary,Base Salary,ADJ Salary,Apps,Min,G,NPG,A,...,Rating,Tackles,Inter,Fouls,Offsides,Clear,Drb_Def,Blocks,AvgP,PS%
Forward,<35,2018.6875,31.125,100999.525,5251975.0,5692077.95,29.05,1925.825,8.1375,7.0625,2.9375,...,6.758889,0.631384,0.235737,0.861372,0.011337,0.523813,0.498469,0.060069,18.9974,72.580483


In [83]:
rooney2017[r.columns]

,year,Age,Weekly Salary,Base Salary,ADJ Salary,Apps,Min,G,NPG,A,...,Rating,Tackles,Inter,Fouls,Offsides,Clear,Drb_Def,Blocks,AvgP,PS%
1890,2017,31,160000.0,8320000.0,9457906.0,31.0,2301.0,10.0,7.0,2.0,...,6.635161,0.903226,0.516129,1.129032,0.0,1.16129,1.032258,0.193548,37.483871,77.710843
